<a href="https://colab.research.google.com/github/EWOLO-JM/Master-M2-CIS/blob/main/CodeMethodeCISLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dissimulation du message**

In [ ]:
import cv2
import numpy as np
from google.colab import files
import io

def normalize_image(image, target_size=(512, 512)):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def cover_image_preparation(cover_picture, m, n):
    cover_segments = []
    for i in range(0, cover_picture.shape[0], m):
        for j in range(0, cover_picture.shape[1], n):
            segment = cover_picture[i:i+m, j:j+n]
            lowest_value = np.min(segment)
            highest_value = np.max(segment)
            cover_segments.append((lowest_value, highest_value))
    return cover_segments

def secret_data_preparation(secret_data, num_segments):
    binary_secret_data = ''.join(format(ord(char), '07b') for char in secret_data)
    # Ajouter des zéros à la fin pour atteindre la longueur requise
    while len(binary_secret_data) < num_segments * 2:  # Multiplié par 2 pour prendre en compte les deux bits par segment
        binary_secret_data += '0'
    return binary_secret_data

def mapping(cover_segments, binary_secret_data, mapping_key):
    mapping_flags = []
    for i, (lowest, highest) in enumerate(cover_segments):
        mapped_bit = int(not (int(binary_secret_data[i]) ^ ((highest & 128) >> 7)))
        mapping_flags.append(mapped_bit)
    return mapping_flags

def embed_message(cover_image, secret_data, m, n, mapping_key):
    # Normaliser l'image à la taille spécifiée (512x512)
    normalized_cover_image = normalize_image(cover_image)
    cover_segments = cover_image_preparation(normalized_cover_image, m, n)

    # Ajouter le nombre total de segments comme argument pour la préparation des données secrètes
    binary_secret_data = secret_data_preparation(secret_data, len(cover_segments))

    mapping_flags = mapping(cover_segments, binary_secret_data, mapping_key)

    stego_image = np.copy(normalized_cover_image)

    for i, (lowest, highest) in enumerate(cover_segments):
        row = i // (normalized_cover_image.shape[1] // n)
        col = i % (normalized_cover_image.shape[1] // n)
        stego_image[row * m:(row + 1) * m, col * n:(col + 1) * n] = (stego_image[row * m:(row + 1) * m, col * n:(col + 1) * n] & ~1) | mapping_flags[i]

    return stego_image

# Charger l'image de couverture depuis l'ordinateur local
uploaded = files.upload()
cover_image_path = list(uploaded.keys())[0]
cover_image_data = uploaded[cover_image_path]
cover_image = cv2.imdecode(np.frombuffer(cover_image_data, np.uint8), cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image a réussi
if cover_image is None:
    print("Erreur: Impossible de lire l'image de couverture.")
else:
    # Données secrètes à dissimuler
    secret_data = "Hello Mr EWOLO "

    # Paramètres pour la méthode CIHLHF
    m = 4
    n = 4
    mapping_key = [7, 8, 1, 2, 3, 4, 6, 5]

    # Dissimuler le message dans l'image de couverture
    stego_image = embed_message(cover_image, secret_data, m, n, mapping_key)

    # Enregistrer l'image stéganographique avec un nom différent à chaque itération
    for i in range(1, 2):  # Changer la plage selon le nombre d'itérations souhaité
        stego_image_path = f"/content/stego_image_normalized_{i}.jpg"
        cv2.imwrite(stego_image_path, stego_image)
        print(f"Image stéganographique enregistrée : {stego_image_path}")


***Affichage de l'image stégo***

In [ ]:
import cv2
from matplotlib import pyplot as plt

# Charger l'image stéganographique
stego_image_path = "/content/stego_image_normalized_1.jpg"
stego_image = cv2.imread(stego_image_path, cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image a réussi
if stego_image is None:
    print("Erreur: Impossible de lire l'image stéganographique.")
else:
    # Afficher l'image stéganographique
    plt.imshow(stego_image, cmap='gray')
    plt.title('Image stéganographique')
    plt.axis('off')
    plt.show()


***Afficher les dimmenssions de l'image stégo***

In [ ]:
print("Dimensions de l'image stéganographique:", stego_image.shape)

# **Extraction du message secret**

In [ ]:
import cv2
import numpy as np
from google.colab import files

def is_image_normalized(image, target_size=(512, 512)):
    return image.shape[:2] == target_size

def cover_image_preparation(cover_picture, m, n):
    cover_segments = []
    for i in range(0, cover_picture.shape[0], m):
        for j in range(0, cover_picture.shape[1], n):
            segment = cover_picture[i:i+m, j:j+n]
            lowest_value = np.min(segment)
            highest_value = np.max(segment)
            cover_segments.append((lowest_value, highest_value))
    return cover_segments

def extract_message(stego_image, m, n, mapping_key):
    # Vérifier si l'image stéganographique est normalisée en 512x512
    if not is_image_normalized(stego_image):
        print("Erreur: L'image stéganographique n'est pas normalisée en 512x512.")
        return None

    # Utiliser la fonction cover_image_preparation sur l'image stéganographique
    stego_segments = cover_image_preparation(stego_image, m, n)

    extracted_binary_data = ""
    for i, (lowest, highest) in enumerate(stego_segments):
        row = i // (stego_image.shape[1] // n)
        col = i % (stego_image.shape[1] // n)

        # Extraire le bit du bit de poids faible de l'image stéganographique
        extracted_bit = (stego_image[row * m:(row + 1) * m, col * n:(col + 1) * n] & 1)[0, 0]

        # Appliquer le XOR avec le bit du bit de poids fort du segment
        extracted_bit ^= ((highest & 128) >> 7)

        extracted_binary_data += str(extracted_bit)

    # Convertir les données binaires extraites en une chaîne Unicode avec gestion des erreurs
    try:
        unicode_data = bytes(int(extracted_binary_data[i:i+8], 2) for i in range(0, len(extracted_binary_data), 8)).decode("utf-8", errors="ignore")
        return unicode_data
    except ValueError as e:
        print(f"Erreur de conversion vers UTF-8 : {e}")
        return None

# Charger l'image stéganographique depuis l'ordinateur local
uploaded = files.upload()
stego_image_path = list(uploaded.keys())[0]
stego_image_data = uploaded[stego_image_path]
stego_image = cv2.imdecode(np.frombuffer(stego_image_data, np.uint8), cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image stéganographique a réussi
if stego_image is None:
    print("Erreur: Impossible de lire l'image stéganographique.")
else:
    # Paramètres pour la méthode CIHLHF
    m = 4
    n = 4
    mapping_key = [7, 8, 1, 2, 3, 4, 6, 5]

    # Extraire le message secret de l'image stéganographique
    extracted_message = extract_message(stego_image, m, n, mapping_key)

    if extracted_message is not None:
        # Afficher le message extrait
        print("Message secret extrait:", extracted_message)
    else:
        print("Extraction du message échouée.")


In [ ]:
import cv2
from matplotlib import pyplot as plt

# Charger l'image stéganographique
stego_image_path = "/content/stego_image_normalized_11.jpg"
stego_image = cv2.imread(stego_image_path, cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image a réussi
if stego_image is None:
    print("Erreur: Impossible de lire l'image stéganographique.")
else:
    # Afficher l'image stéganographique
    plt.imshow(stego_image, cmap='gray')
    plt.title('Image stéganographique')
    plt.axis('off')
    plt.show()


***Affichage les dimenssions du méssage secret***

In [ ]:
print("Dimensions de l'image stéganographique:", stego_image.shape)

Importer un fichier

In [ ]:
from google.colab import files
file = files.upload()

# **Métriques d'évaluation**

**1. Distorsion Visuelle (MSE et PSNR)**

Calculez la MSE entre l'image originale et l'image stéganographique.
Calculez le PSNR en utilisant la formule PSNR = 10 * log10(max^2 / MSE), où max est la valeur maximale possible pour un pixel.

In [ ]:
import cv2
import numpy as np

# Chargez l'image originale et l'image stéganographique
original_image = cv2.imread("original_image.jpg", cv2.IMREAD_GRAYSCALE)
stego_image = cv2.imread("stego_image.jpg", cv2.IMREAD_GRAYSCALE)

# Calculez la MSE
mse = np.sum((original_image - stego_image)**2) / float(original_image.size)

# Calculez le PSNR
max_pixel_value = 255  # Pour une image en niveaux de gris
psnr = 10 * np.log10((max_pixel_value**2) / mse)

print(f"MSE: {mse}, PSNR: {psnr}")


**2. Capacité de Charge :**



Calculez la capacité de charge en bits par pixel (bpp) ou en pourcentage de la capacité totale de l'image.

In [ ]:
# Calcul de la capacité de charge en bpp
capacity_bpp = (original_image.size * 2) / float(original_image.size)

# Calcul de la capacité de charge en pourcentage
capacity_percentage = (capacity_bpp / (original_image.size * 8)) * 100

print(f"Capacité de charge (bpp): {capacity_bpp}, Capacité de charge (%): {capacity_percentage}%")


**3. Robustesse :**


Appliquez des attaques à l'image stéganographique et évaluez la distorsion visuelle.

In [ ]:
# Appliquez une attaque (par exemple, filtrage)
filtered_stego_image = cv2.GaussianBlur(stego_image, (5, 5), 0)

# Calculez la MSE et le PSNR après l'attaque
mse_filtered = np.sum((original_image - filtered_stego_image)**2) / float(original_image.size)
psnr_filtered = 10 * np.log10((max_pixel_value**2) / mse_filtered)

print(f"MSE après filtrage: {mse_filtered}, PSNR après filtrage: {psnr_filtered}")


# **Résultats Expérimentaux et Comparaison**

**Capacité de dissimulation**

In [ ]:
import matplotlib.pyplot as plt

# Données
methodes = ['CISFSA', 'CISMSB', 'CISLHF']
valeurs = [6272, 16384, 49152]

# Couleurs pour chaque bande
couleurs = ['blue', 'orange', 'purple']

# Création de l'histogramme
plt.bar(methodes, valeurs, color=couleurs)

# Titres et labels
plt.title('Comparaison des Méthodes de Dissimulation sans Couverture')
plt.xlabel('Méthodes')
plt.ylabel('Capacité de dissimulation')

# Limite de l'axe des ordonnées
plt.ylim(0, 60000)

# Affichage de l'histogramme
plt.show()
